# 中文垃圾邮件分类

## 1.数据清洗

In [1]:
import string
import re
import gensim
import jieba

C:\Users\HJQ\Anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
def get_data():
    '''
    获取数据
    :return: 文本数据，对应的labels
    '''
    with open("ham_5000.utf8", encoding="utf8") as ham_f, open("spam_5000.utf8", encoding="utf8") as spam_f:
        ham_data = ham_f.readlines()
        spam_data = spam_f.readlines()
 
        ham_label = np.ones(len(ham_data)).tolist()
        spam_label = np.zeros(len(spam_data)).tolist()
 
        corpus = ham_data + spam_data
 
        labels = ham_label + spam_label
 
    return corpus, labels

In [3]:
#取空文件
def remove_empty_docs(corpus, labels):
    filtered_corpus = []
    filtered_labels = []
    for docs, label in zip(corpus, labels):
        #移除字符串头尾指定的字符(默认为空格)
        if docs.strip():
            filtered_corpus.append(docs)
            filtered_labels.append(label)
    return filtered_corpus, filtered_labels

In [4]:
# 加载停用词
with open("cnstopwords.txt", encoding="utf8") as f:
    stopword_list = f.readlines()


def tokenize_text(text):
    tokens = jieba.cut(text)
    tokens = [token.strip() for token in tokens]
    return tokens

In [5]:
#去除特殊符号
def remove_special_characters(text):
    tokens = tokenize_text(text)
    pattern = re.compile('[{}]'.format(re.escape(string.punctuation)))
    filtered_tokens = filter(None, [pattern.sub('', token) for token in tokens])
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

In [6]:
#去除英文字母
def remove_english_characters(text):
    tokens = tokenize_text(text)
    pattern = re.compile('[a-zA-Z]'.format(re.escape(string.punctuation)))
    filtered_tokens = filter(None, [pattern.sub('', token) for token in tokens])
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

In [7]:
#去除数字和空格
def remove_numbers(text):
    tokens = tokenize_text(text)
    pattern = re.compile('[\d]'.format(re.escape(string.punctuation)))
    filtered_tokens = filter(None, [pattern.sub('', token) for token in tokens])
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

In [8]:
#去除停用词
def remove_stopwords(text):
    tokens = tokenize_text(text)
    filtered_tokens = [token for token in tokens if token not in stopword_list]
    filtered_text = ''.join(filtered_tokens)
    return filtered_text


def normalize_corpus(corpus, tokenize=False):
    normalized_corpus = []
    for text in corpus:

        text = remove_special_characters(text)
        text = remove_stopwords(text)
        normalized_corpus.append(text)
        if tokenize:
            text = tokenize_text(text)
            normalized_corpus.append(text)

    return normalized_corpus

## 2.特征提取

In [9]:
'''提取特征'''
from sklearn.feature_extraction.text import CountVectorizer

def bow_extractor(corpus, ngram_range=(1, 1)):
    vectorizer = CountVectorizer(min_df=1, ngram_range=ngram_range)
    features = vectorizer.fit_transform(corpus)
    return vectorizer, features


from sklearn.feature_extraction.text import TfidfTransformer

def tfidf_transformer(bow_matrix):
    transformer = TfidfTransformer(norm='l2',
                                   smooth_idf=True,
                                   use_idf=True)
    tfidf_matrix = transformer.fit_transform(bow_matrix)
    return transformer, tfidf_matrix


from sklearn.feature_extraction.text import TfidfVectorizer

def tfidf_extractor(corpus, ngram_range=(1, 1)):
    vectorizer = TfidfVectorizer(min_df=1,
                                 norm='l2',
                                 smooth_idf=True,
                                 use_idf=True,
                                 ngram_range=ngram_range)
    features = vectorizer.fit_transform(corpus)
    return vectorizer, features

## 3.模型训练

In [10]:
def train_predict_evaluate_model(classifier,
                                 train_features, train_labels,
                                 test_features, test_labels):
    # build model
    classifier.fit(train_features, train_labels)
    # predict using model
    predictions = classifier.predict(test_features)
    # evaluate model prediction performance
    get_metrics(true_labels=test_labels,
                predicted_labels=predictions)
    return predictions

In [11]:
from sklearn import metrics

def get_metrics(true_labels, predicted_labels):
    print('准确率:', np.round(
        metrics.accuracy_score(true_labels,
                               predicted_labels),
        2))
    print('精度:', np.round(
        metrics.precision_score(true_labels,
                                predicted_labels,
                                average='weighted'),
        2))
    print('召回率:', np.round(
        metrics.recall_score(true_labels,
                             predicted_labels,
                             average='weighted'),
        2))
    print('F1得分:', np.round(
        metrics.f1_score(true_labels,
                         predicted_labels,
                         average='weighted'),
        2))

In [12]:
import numpy as np
from sklearn.model_selection import train_test_split

corpus, labels=get_data()  
#获取数据集
print("总的数据量：",len(labels))
corpus,labels=remove_empty_docs(corpus,labels)
print("样本之一：",corpus[10])
label_name_map=["垃圾邮件","正常邮件"]
print("实际：",label_name_map[int(labels[10])],label_name_map[int(labels[5900])])      
#对数据进行划分
train_corpus, test_corpus, train_labels, test_labels = train_test_split(corpus, labels,test_size=0.3)

总的数据量： 10001
样本之一： 北京售票员可厉害，嘿嘿，有专座的，会直接拉着脖子指着鼻子让上面的人站起来让 座的，呵呵，比较赞。。。 杭州就是很少有人给让座，除非司机要求乘客那样做。 五一去杭州一个景点玩，车上有两个不到一岁的小孩，就是没有人给让座，没办法家长只能在车上把小孩的推车打开让孩子坐进去，但是孩子还是闹，只能抱着，景点离市区很远，车上很颠，最后家长坐在地上抱孩子，就是没有一个人给让座，要是在北京，一上车就有人让座了

实际： 正常邮件 垃圾邮件


In [13]:
corpus[0:5]

['讲的是孔子后人的故事。一个老领导回到家乡，跟儿子感情不和，跟贪财的孙子孔为本和睦。 老领导的弟弟魏宗万是赶马车的。 有个洋妞大概是考察民俗的，在他们家过年。 孔为本总想出国，被爷爷教育了。 最后，一家人基本和解。 顺便问另一类电影，北京青年电影制片厂的。中越战背景。一军人被介绍了一个对象，去相亲。女方是军队医院的护士，犹豫不决，总是在回忆战场上负伤的男友，好像还没死。最后 男方表示理解，归队了。\n',
 '不至于吧，离开这个破公司就没有课题可以做了？ 谢谢大家的关心，她昨天晚上睡的很好。MM她自己已经想好了。见机行事吧，拿到相关的能出来做论文的材料，就马上辞职。 唉！看看吧，说不定还要各为XDJM帮出出找工作的主意呢。MM学通信的，哈尔滨工程大学的研究生，不想在哈碌碌无为的做设计，因此才出来的。先谢谢了啊。！！！ 本人语文不好，没加标点。辛苦那些看不懂的XDJM么了。\n',
 '生一个玩玩，不好玩了就送人 第一，你要知道，你们恋爱前，你爹妈对她是毫无意义的。没道理你爹妈就要求她生孩子，她就得听话。换句话说，你岳父母要未来孩子跟妈姓，你做的到吗？夫妻是平等的。如果你没办法答应岳父母，她干吗答应你爹妈呢？ 第二，有了孩子你养不养的起？不是说想生就生，图你爹妈一个高兴，如果没有房子，没有充足的财力，生孩子只会带给你们更多的困难，生小孩容易，养小孩难啊。\n',
 '微软中国研发啥？本地化？ 新浪科技讯 8月24日晚10点，微软中国对外宣布说，在2006财年(2005年7月-2006年6月)，公司将在中国招聘约800名新员工。 其中，一半以上的新聘人员将为研发人员，其他将是销售、市场和服务人员。同时，有近300个职位将面向新毕业的大学本科生、硕士研究生、MBA和博士生。 在2005财年，微软在中国的业务取得了骄人成绩，成为微软全球增长速度最快的子公司之一。\n',
 '要是他老怕跟你说话耽误时间 你可得赶紧纠正他这个观点 标  题: Re: 今天晚上的事情，有点郁闷 这个...其实以前有问题的时候都是当面解决，后来他说你有什么想不通的可以到板上去 问问别人，然后你就知道是谁不对了，所以这次我就来问，我觉得挺好，避免正面冲突， 他最怕耽误他时间，这样正好也不耽误他时间，也解了我的心结 : 感觉这两人都不够坦诚 : gg郁闷了就找mm别扭 : mm别扭了就到版上哭诉 -

In [14]:
norm_train_corpus=normalize_corpus(train_corpus)
norm_test_corpus=normalize_corpus(test_corpus)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\HJQ\AppData\Local\Temp\jieba.cache
Loading model cost 1.432 seconds.
Prefix dict has been built successfully.


In [15]:
#词袋模型特征
bow_vectorizer,bow_train_features=bow_extractor(norm_train_corpus)
bow_test_features=bow_vectorizer.transform(norm_test_corpus)

#tfidfT特征
tfidf_vectorizer,tfidf_train_features=tfidf_extractor(norm_train_corpus)
tfidf_test_features=tfidf_vectorizer.transform(norm_test_corpus)

#tokenize documents
tokenized_train=[jieba.lcut(text)  for text in norm_train_corpus]
print(tokenized_train[2:10])
tokenized_test=[jieba.lcut(text)  for text in norm_test_corpus]
print(tokenized_test[2:10])

[['尊敬', '的', '公司', '您好', '！', '打扰', '之处', '请', '见谅', '！', '我', '深圳', '公司', '愿', '在', '互惠互利', '、', '诚信为本', '代开', '3', '厘', '2', '点', '国税', '、', '地税', '等', '发票', '。', '增值税', '和', '海关', '缴款', '书', '就', '以', '2', '点', '7', '点来', '代开', '。', '手机', '：', '13510631209', '联系人', '：', '邝', '先生', '邮箱', '：', 'ao998163com', '祥细', '资料', '合作', '告知', '，', '希望', '合作', '。', '谢谢', '！', '！'], ['您好', '：', '如此', '条', '信息', '对', '您', '的', '打扰', '表示歉意', '，', '并', '请谅解', '。', '现我司', '有', '：', '商品销售', '、', '运输', '、', '建筑', '、', '广告', '等', '发票', '可代开', '，', '点数', '低', '，', '如需', '请', '电', '：', '13927401103', '刘生'], ['棒', '老公', '成人', '用品', '网', '，', '为', '您', '提供', '国内', '最新', '最', '实惠', '的', '产品', '用品', '！', '网址', '：', 'httpwww8919470com'], ['发信人', 'PMHT', '加油', '啊', '信区', 'DEE', '标题', '急寻', '明日', '出差', '去', '深圳', '的', '同学', '老师', '帮忙', '最好', '是', '中午', '或', '上午', '航班', '的', '，', '帮', '我', '捎带', '一', '重要', '证件', '前往', '深圳', '，', '对方', '在', '机场', '候着', '！', '！', '！', '由于', '是', '明天', '需要', '该', '证件', '，', '所以', '快递

## 4.模型结果

In [16]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression

mnb = MultinomialNB()
svm = SGDClassifier(loss='hinge')
lr = LogisticRegression()
# 基于词袋模型的多项朴素贝叶斯
print("基于词袋模型特征的贝叶斯分类器")
mnb_bow_predictions = train_predict_evaluate_model(classifier=mnb,
                                                   train_features=bow_train_features,
                                                   train_labels=train_labels,
                                                   test_features=bow_test_features,
                                                   test_labels=test_labels)

# 基于词袋模型特征的逻辑回归
print("基于词袋模型特征的逻辑回归")
lr_bow_predictions = train_predict_evaluate_model(classifier=lr,
                                                  train_features=bow_train_features,
                                                  train_labels=train_labels,
                                                  test_features=bow_test_features,
                                                  test_labels=test_labels)

# 基于词袋模型的支持向量机方法
print("基于词袋模型的支持向量机")
svm_bow_predictions = train_predict_evaluate_model(classifier=svm,
                                                   train_features=bow_train_features,
                                                   train_labels=train_labels,
                                                   test_features=bow_test_features,
                                                   test_labels=test_labels)


# 基于tfidf的多项式朴素贝叶斯模型
print("基于tfidf的贝叶斯模型")
mnb_tfidf_predictions = train_predict_evaluate_model(classifier=mnb,
                                                     train_features=tfidf_train_features,
                                                     train_labels=train_labels,
                                                     test_features=tfidf_test_features,
                                                     test_labels=test_labels)
# 基于tfidf的逻辑回归模型
print("基于tfidf的逻辑回归模型")
lr_tfidf_predictions=train_predict_evaluate_model(classifier=lr,
                                                     train_features=tfidf_train_features,
                                                     train_labels=train_labels,
                                                     test_features=tfidf_test_features,
                                                     test_labels=test_labels)


# 基于tfidf的支持向量机模型
print("基于tfidf的支持向量机模型")
svm_tfidf_predictions = train_predict_evaluate_model(classifier=svm,
                                                     train_features=tfidf_train_features,
                                                     train_labels=train_labels,
                                                     test_features=tfidf_test_features,
                                                     test_labels=test_labels)

基于词袋模型特征的贝叶斯分类器
准确率: 0.96
精度: 0.96
召回率: 0.96
F1得分: 0.96
基于词袋模型特征的逻辑回归
准确率: 0.95
精度: 0.95
召回率: 0.95
F1得分: 0.95
基于词袋模型的支持向量机
准确率: 0.96
精度: 0.96
召回率: 0.96
F1得分: 0.96
基于tfidf的贝叶斯模型
准确率: 0.95
精度: 0.96
召回率: 0.95
F1得分: 0.95
基于tfidf的逻辑回归模型
准确率: 0.93
精度: 0.94
召回率: 0.93
F1得分: 0.93
基于tfidf的支持向量机模型
准确率: 0.96
精度: 0.96
召回率: 0.96
F1得分: 0.96


In [17]:
import re

label_name_map=["垃圾邮件","正常邮件"]
num = 0
for document, label, predicted_label in zip(test_corpus, test_labels, svm_tfidf_predictions):
      if label == 0 and predicted_label == 0:
        print('邮件类型:', label_name_map[int(label)])
        print('预测的邮件类型:', label_name_map[int(predicted_label)])
        print('文本:-')
        print(re.sub('\n', ' ', document))

        num += 1
        if num == 4:
            break

num = 0
for document, label, predicted_label in zip(test_corpus, test_labels, svm_tfidf_predictions):
     if label == 1 and predicted_label == 0:
        print('邮件类型:', label_name_map[int(label)])
        print('预测的邮件类型:', label_name_map[int(predicted_label)])
        print('文本:-')
        print(re.sub('\n', ' ', document))

        num += 1
        if num == 4:
            break

邮件类型: 垃圾邮件
预测的邮件类型: 垃圾邮件
文本:-
贵公司负责人：你好！ 本公司(祥泰实业有限公司）具有进出口及国内贸易的企业, 承多家公司委托:有广告/建筑工程/其它服务/商品销售/...等的 发票向外代开.点数优惠,本公司原则是满意后付款.有意者请来 电洽谈. 电  话：013631690076 邮  箱：shitailong8@163.com 联系人：郭 生 如给贵公司带来不便请谅解! 
邮件类型: 垃圾邮件
预测的邮件类型: 垃圾邮件
文本:-
深圳市顺发电子(美国)有限公司 网址:http://www.mgqt.com 公司产品有:购买窃听器．手机窃听器．美国监听王. GSM_8移动电话拦截系统．电子变牌. 电表控制器zx003．电话变声器．游戏机反遥控. 隐形汽车牌照as240．等实惠产品： (24)小时订购电话：0755-61199926 (24)小时联系人：陈先生 
邮件类型: 垃圾邮件
预测的邮件类型: 垃圾邮件
文本:-
你好： 真正自己国人的，免费注册，有偿回报的系统！ 通宝：最佳网络伴侣 闲余间还能为你增加收入！！ 绿色软件，注册简单，而且免费： Http://rich138.008.net 赠送《超高能潜意识CD》 只需购买以下课程200元，即赠送一套！！ Http://www.cg-biz.com/list.txt 
邮件类型: 垃圾邮件
预测的邮件类型: 垃圾邮件
文本:-
您好： 我是广州市实达贸易有限公司,现有剩余(广告.运输.服务.商品.) 等各种普通发票可以代开,只收2% 的税点 联 人: 张高伟 手 机:  13828415779              (先验票后付款) 电 话:  020-33622299 回 邮:  zz8cle@avl.com.cn 
邮件类型: 正常邮件
预测的邮件类型: 垃圾邮件
文本:-
极度yy啊~~ 前言：已经很久没有熬夜了，深夜总是我们敞开灵魂的时刻。温温柔柔的写了一个东东给网友mm寄过去，不知道她会不会误会，以为我喜欢她。希望大家给点意见。 琳，你好： 我的手机已经发光包月，现在只能发信问候。其实写信也是和有情致的事情。只是现代的科技把我们推向了平庸和无奇。我不喜欢没有变化的世界，我不喜欢没有期待的日立，我不喜欢没有云雨的天际，我不喜欢没有信札的联系。所以，我给你写信。 
邮件类

## 5.探索性方法

In [18]:
#词袋模型
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
# CountVectorizer类会把文本全部转换为小写，然后将文本词块化。主要是分词，分标点
data_train_cnt = vectorizer.fit_transform(train_corpus)
data_test_cnt = vectorizer.transform(test_corpus)

In [19]:
#变成TF-IDF矩阵
from sklearn.feature_extraction.text import TfidfTransformer

transformer = TfidfTransformer()
data_train_tfidf = transformer.fit_transform(data_train_cnt)
data_test_tfidf = transformer.transform(data_test_cnt)

In [20]:
# 随机森林
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=0, n_estimators=100, max_depth=None, verbose=0, n_jobs=-1)
rf.fit(data_train_tfidf, train_labels)
score = rf.score(data_test_tfidf, test_labels)
print("RF score: ", score)

RF score:  0.8760413195601466


In [21]:
# LightGBM的方法
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV

lgb_clf = lgb.LGBMClassifier()
# lgb_clf.fit(data_train_tfidf, label_train)
# 使用网格搜索得到适当参数
param_test = {
    'max_depth': range(2, 3)
}
gsearch = GridSearchCV(estimator=lgb_clf, param_grid=param_test, scoring='roc_auc', cv=5)
gsearch.fit(data_train_tfidf,train_labels)
# print(gsearch.best_params_)
score = gsearch.score(data_test_tfidf, test_labels)
print("LGBM score: ", score)

LGBM score:  0.9726179655785464


In [22]:
# XGBoost方法
import xgboost as xgb

xgb_clf = xgb.XGBClassifier(n_estimators=100, n_jobs=-1, max_depth=15, min_child_weight=3, colsample_bytree=0.4)
xgb_clf.fit(data_train_tfidf,train_labels)
score = xgb_clf.score(data_test_tfidf, test_labels)
print("XGBoost score: ", score)

C:\Users\HJQ\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:10:12] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBoost score:  0.9596801066311229
